## Import library

In [1]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)

import data_load
import numpy as np
import pandas as pd
import logging
import incremental_ps_score_estimator as ipse
import math
import timeit
import utils
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import KFold

import dowhy.datasets
import dowhy
from dowhy import CausalModel
import matplotlib.pyplot as plt

from econml.drlearner import ForestDRLearner, LinearDRLearner
from econml.metalearners import SLearner, XLearner, TLearner
from econml.ortho_forest import CausalTree, ContinuousTreatmentOrthoForest, DiscreteTreatmentOrthoForest
from econml.dml import ForestDMLCateEstimator, LinearDMLCateEstimator, SparseLinearDMLCateEstimator
from econml.inference import BootstrapInference
from econml.sklearn_extensions.linear_model import WeightedLasso, WeightedLassoCV

### Import sklearn
from scipy.stats import sem
import scipy.stats as st
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import explained_variance_score, mean_squared_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.linear_model import LassoCV, ElasticNetCV
from cforest.forest import CausalForest


/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


## Load data

In [2]:
file_path = "https://msalicedatapublic.blob.core.windows.net/datasets/Pricing/pricing_sample.csv"
train_data = pd.read_csv(file_path)

In [3]:
train_data.price.value_counts()

1.0    4346
0.8    3089
0.9    2565
Name: price, dtype: int64

In [4]:
train_data['treatment'] = np.where(train_data['price'] == 1, 1, 0)
train_data['price'] = np.where(train_data['price'] == 1, 1, 0.85)

In [5]:
train_data.price.value_counts()

0.85    5654
1.00    4346
Name: price, dtype: int64

In [6]:
train_data.head()

,account_age,age,avg_hours,days_visited,friends_count,has_membership,is_US,songs_purchased,income,price,demand,treatment
0,3,53,1.834234,2,8,1,1,4.903237,0.960863,1.00,3.917117,1
1,5,54,7.171411,7,9,0,1,3.330161,0.732487,1.00,11.585706,1
2,3,33,5.351920,6,9,0,1,3.036203,1.130937,1.00,24.675960,1
3,2,34,6.723551,0,8,0,1,7.911926,0.929197,1.00,6.361776,1
4,4,30,2.448247,5,8,1,0,7.148967,0.533527,0.85,12.624123,0


## Features engineering

In [7]:
outcome = "demand"
treatment = "treatment"
col = list(train_data.columns)
col.remove("price")
print(col)

cov = col[:]
cov.remove(treatment)
cov.remove(outcome)
cov.remove('income')
print(cov)

features = col[:]
features.remove(outcome)

print(features)

['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased', 'income', 'demand', 'treatment']
['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased']
['account_age', 'age', 'avg_hours', 'days_visited', 'friends_count', 'has_membership', 'is_US', 'songs_purchased', 'income', 'treatment']


In [8]:
train_data.treatment.value_counts()

0    5654
1    4346
Name: treatment, dtype: int64

In [9]:
# Get test data
X_test = np.linspace(0, 5, 100).reshape(-1, 1)
X_test_data = pd.DataFrame(X_test, columns=["income"])

## Synthesis function

In [10]:
# Define underlying treatment effect function given DGP
def gamma_fn(X):
    return -3 - 14 * (X["income"] < 1)

def beta_fn(X):
    return 20 + 0.5 * (X["avg_hours"]) + 5 * (X["days_visited"] > 4)

def demand_fn(data, T):
    Y = gamma_fn(data) * T + beta_fn(data)
    return Y

def true_te(x, n, stats):
    if x < 1:
        subdata = train_data[train_data["income"] < 1].sample(n=n, replace=True)
    else:
        subdata = train_data[train_data["income"] >= 1].sample(n=n, replace=True)
    te_array = subdata["price"] * gamma_fn(subdata) / (subdata["demand"])
    if stats == "mean":
        return np.mean(te_array)
    elif stats == "median":
        return np.median(te_array)
    elif isinstance(stats, int):
        return np.percentile(te_array, stats)

In [11]:
# Get the estimate and range of true treatment effect
truth_te_estimate = np.apply_along_axis(true_te, 1, X_test, 1000, "mean")  # estimate
truth_te_upper = np.apply_along_axis(true_te, 1, X_test, 1000, 95)  # upper level
truth_te_lower = np.apply_along_axis(true_te, 1, X_test, 1000, 5)  # lower level

In [12]:
print("Estimation true effect {}".format(np.mean(truth_te_estimate)))
print("Upper bound true effect {}".format(np.mean(truth_te_upper)))
print("Lower bound true effect {}".format(np.mean(truth_te_lower)))

Estimation true effect -0.45369095264802645
Upper bound true effect -0.2795216649663177
Lower bound true effect -0.75770637353686


In [13]:
te_array = train_data["price"] * gamma_fn(train_data) / (train_data["demand"])
true_effect = np.mean(te_array)

## Estimation effect with incremental propensity score

In [14]:
features

['account_age',
 'age',
 'avg_hours',
 'days_visited',
 'friends_count',
 'has_membership',
 'is_US',
 'songs_purchased',
 'income',
 'treatment']

In [15]:
x = cov.copy()
x.append(treatment)
x

['account_age',
 'age',
 'avg_hours',
 'days_visited',
 'friends_count',
 'has_membership',
 'is_US',
 'songs_purchased',
 'treatment']

In [16]:
## Fit treatment
model_t = LogisticRegression()
model_t.fit(train_data[cov], train_data[treatment])

train_data['p1'] = model_t.predict_proba(train_data[cov])[:,1]
train_data['p0'] = 1 - train_data['p1']

train_data["prediction"] = np.where(train_data["p1"] >= 0.5, 1, 0)
acc = accuracy_score(train_data["prediction"], train_data["treatment"])
print("Accuracy score of train data {}".format(acc))

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(train_data[features], train_data[outcome])

Accuracy score of train data 0.5654


GradientBoostingRegressor(n_estimators=5000, random_state=0)

In [17]:
delta = -0.015843108
influence = ipse.influence_function(train_data, treatment, cov, outcome, features, delta, model_y, model_t)
means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

print("Estimation effect {:.2f}".format(means_incre))
print("MAE {:.2f}".format(utils.abs_ate(true_effect, means_incre)))

Estimation effect -0.94
MAE 0.01


### Estimation effect in train-test

In [18]:
train, test = train_test_split(train_data, test_size=0.1, random_state = 1)

## Fit treatment
model_t = LogisticRegression()
model_t.fit(train[cov], train[treatment])

train['p1'] = model_t.predict_proba(train[cov])[:,1]
train['p0'] = 1 - train['p1']

train["prediction"] = np.where(train["p1"] >= 0.5, 1, 0)
acc = accuracy_score(train["prediction"], train["treatment"])
print("Accuracy score of train data {}".format(acc))

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(train[features], train[outcome])


/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

Accuracy score of train data 0.5644444444444444


GradientBoostingRegressor(n_estimators=5000, random_state=0)

In [19]:
delta = 2.0

te_array = test["price"] * gamma_fn(test) / (test["demand"])
true_effect_test = np.mean(te_array)

te_array = train["price"] * gamma_fn(train) / (train["demand"])
true_effect_train = np.mean(te_array)

influence = ipse.influence_function(train, treatment, cov, outcome, features, delta, model_y, model_t)
means_incre_train, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

influence = ipse.influence_function(test, treatment, cov, outcome, features, delta, model_y, model_t)
means_incre_test, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

origin_mae_train = utils.abs_ate(true_effect_train, means_incre_train)
origin_mae_test = utils.abs_ate(true_effect_test, means_incre_test)

# print("Estimation effect {:.2f}".format(means_incre))
# print("MAE {:.2f}".format(utils.abs_ate(true_effect, means_incre)))

/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib/incremental_ps_score_estimator.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['p1'] = model_t.predict_proba(data[covariate])[:,1]
/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib/incremental_ps_score_estimator.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['p0'] = 1 - data['p1']
/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib/incremental_ps_score_estimator.py:

In [20]:
print(origin_mae_train, origin_mae_test)

0.7068891358439671 0.739408703015396


## Optimization with single delta

In [23]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence), influence

def optimization(data, true_effect):
    threhold = tf.constant([0.001])
    delta = tf.Variable(np.random.randint(low=1, high=40, size=1), 
                        trainable = True, 
                        dtype = tf.float32)
    delta_seq = []
    losses = []
    effects = []
    influences = []
    for i in tqdm(range(100000)):
        with tf.GradientTape() as tape:
            mu_influence, influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - mu_influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping 
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
            delta_seq.append(delta.numpy())
            losses.append(loss.numpy())
            effects.append(mu_influence.numpy())
            influences.append(influence.numpy())
        if i % 3000 == 0:
            print("Epoch {}. Loss {:.4f}".format(i, loss))
    print("Loss {:.3f}".format(loss))
    print("Effects ", influence)
    return delta, delta_seq, losses, effects, influences

In [ ]:
te_array = train["price"] * gamma_fn(train) / (train["demand"])
true_effect = np.mean(te_array)

delta, delta_seq, losses, effects, influences = optimization(train, true_effect)

  0%|          | 30/100000 [00:00<11:47, 141.23it/s]

Epoch 0. Loss 1.1236


  3%|▎         | 3022/100000 [00:18<10:47, 149.83it/s]

Epoch 3000. Loss 1.1165


  6%|▌         | 6016/100000 [00:38<10:59, 142.58it/s]

...

 21%|██        | 21028/100000 [02:18<08:41, 151.54it/s]

Epoch 21000. Loss 0.9329


 24%|██▍       | 24028/100000 [02:37<07:52, 160.80it/s]

Epoch 24000. Loss 0.7084


 26%|██▌       | 25980/100000 [02:49<08:45, 140.73it/s]

In [ ]:
plt.plot(delta_seq, effects, 'o')
plt.show()

In [ ]:
plt.plot(delta_seq, losses, 'o')
plt.show()

In [ ]:
delta_seq[-1]

In [ ]:
delta_re = delta.numpy()
delta_re = delta_seq[-1]

delta_re

In [ ]:
te_array = test["price"] * gamma_fn(test) / (test["demand"])
true_effect_test = np.mean(te_array)

te_array = train["price"] * gamma_fn(train) / (train["demand"])
true_effect_train = np.mean(te_array)

influence_train = ipse.influence_function(train, treatment, cov, outcome, features, delta_re, model_y, model_t)
means_incre_train, stds_incre = np.mean(influence_train, axis=0), sem(influence, axis=0)

influence_test = ipse.influence_function(test, treatment, cov, outcome, features, delta_re, model_y, model_t)
means_incre_test, stds_incre = np.mean(influence_test, axis=0), sem(influence, axis=0)

optimal_mae_train = utils.abs_ate(true_effect_train, means_incre_train)
optimal_mae_test = utils.abs_ate(true_effect_test, means_incre_test)

## Conclusion

In [ ]:
print("Before Optimization")
print("MAE on training {} and testing {}".format(origin_mae_train, origin_mae_test))
print("After Optimization")
print("MAE on training {} and testing {}".format(optimal_mae_train, optimal_mae_test))


## Confidence interval

In [ ]:
lowers_train, uppers_train = [], []
lowers_test, uppers_test = [], []
means_train, means_test = [], []

for i in tqdm(range(len(delta_seq))):
    train_expr = train.copy()
    test_expr = test.copy()
    influence_train = influences[i]
    delta = delta_seq[i]
    influence_test = ipse.influence_function(test_expr, treatment, cov, outcome, features, delta, model_y, model_t)

    mean_train, stds_incre_train = np.mean(influence_train, axis=0), sem(influence_train, axis=0)
    mean_test, stds_incre_test = np.mean(influence_test, axis=0), sem(influence, axis=0)
    
    lower_train, upper_train = st.t.interval(0.90, 
                                         len(influence_train)-1, 
                                         loc=mean_train, 
                                         scale=stds_incre_train)
    lower_test, upper_test = st.t.interval(0.90, 
                                             len(influence_test)-1, 
                                             loc=mean_test, 
                                             scale=stds_incre_test)
    

    lowers_train.append(lower_train)
    uppers_train.append(upper_train)
    lowers_test.append(lower_test)
    uppers_test.append(upper_test)
    means_train.append(mean_train)
    means_test.append(mean_test)
    


In [ ]:
plt.plot(delta_seq, means_train, label='Estimation effect')
plt.axhline(y=true_effect_train, color='r', linestyle='-', label = "True effect")
plt.fill_between(delta_seq, lowers_train, uppers_train, label="90% BLB CI", alpha=0.3)
plt.ylabel("Treatment Effect")
plt.xlabel("delta")
plt.title("The changes of effects depending on delta")
plt.legend()
plt.show()

In [ ]:
plt.plot(delta_seq, means_test, label='Estimation effect')
plt.axhline(y=true_effect_test, color='r', linestyle='-', label = "True effect")
plt.fill_between(delta_seq, lowers_test, uppers_test, label="90% BLB CI", alpha=0.3)
plt.ylabel("Treatment Effect")
plt.xlabel("delta")
plt.title("The changes of effects depending on delta")
plt.legend()
plt.show()

In [ ]:
------

## K-Fold

In [ ]:
cv = KFold(n_splits=5, random_state=42, shuffle=False)
mae_seq_train = []
mae_seq_test = []

for train_index, test_index in tqdm(cv.split(train_data)):
    df_train, df_test = train_data.loc[train_index, :], train_data.loc[test_index, :]

    model_t = LogisticRegression()
    model_t.fit(train[cov], train[treatment])

    train['p1'] = model_t.predict_proba(train[cov])[:,1]
    train['p0'] = 1 - train['p1']

    train["prediction"] = np.where(train["p1"] >= 0.5, 1, 0)
    acc = accuracy_score(train["prediction"], train["treatment"])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(train[features], train[outcome])

    delta = 0.025128545

    te_array = test["price"] * gamma_fn(test) / (test["demand"])
    true_effect_test = np.mean(te_array)

    te_array = train["price"] * gamma_fn(train) / (train["demand"])
    true_effect_train = np.mean(te_array)

    influence = ipse.influence_function(train, treatment, cov, outcome, features, delta, model_y, model_t)
    means_incre_train, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

    influence = ipse.influence_function(test, treatment, cov, outcome, features, delta, model_y, model_t)
    means_incre_test, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

    origin_mae_train = utils.abs_ate(true_effect_train, means_incre_train)
    origin_mae_test = utils.abs_ate(true_effect_test, means_incre_test)
    
    mae_seq_train.append(origin_mae_train)
    mae_seq_test.append(origin_mae_test)


In [ ]:
means_train, stds_train = np.mean(mae_seq_train, axis=0), sem(mae_seq_train, axis=0)
means_test, stds_test = np.mean(mae_seq_test, axis=0), sem(mae_seq_test, axis=0)

print("Training {} +- {}".format(means_train, stds_train))
print("Testing {} +- {}".format(means_test, stds_test))

## Optimization individual treatment effects

In [ ]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return influence

def optimization(data):
    threhold = tf.constant([0.01])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=10, 
        stddev=10, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = data['mu1'] - data['mu0']
    
    for i in range(50000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.keras.losses.MSE(true_effect, influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=1)
            opt.apply_gradients(zip([d_delta], [delta]))
            print(loss)
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

In [ ]:
te_array = train["price"] * gamma_fn(train) / (train["demand"])
true_effect = np.mean(te_array)

delta, loss = optimization(train, true_effect)

## Optimization with delta list 

In [ ]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data):
    threhold = tf.constant([0.001])
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=0, 
                          maxval=3, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    
    for i in range(50000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
        if i % 1000 == 0:
            print("Epoch: {}. Loss: {:.4f}".format(i, loss))
        if tf.math.less(loss, threhold):
            print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
            break
            
        """
        policy = np.where(data['q1'] >= 0.5, 1, 0)
        rev1 = np.mean(revenue_fn(data = data, 
                          discount_level1 = 0, 
                          discount_level2 = 0.1, 
                          baseline_T = 1,
                          policy = policy))
        print("Revenue {:.2f}".format(rev1))
        """
    return delta


In [ ]:
delta = optimization(train_data)

In [ ]:
delta.shape

In [ ]:
delta = delta.numpy()

In [ ]:
train['p1'] = model_t.predict_proba(train[cov])[:,1]
test['p1'] = model_t.predict_proba(test[cov])[:,1]

te_array = test["price"] * gamma_fn(test) / (test["demand"])
true_effect = np.mean(te_array)

treated_neighbors = (
        NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
        .fit(train['p1'].values.reshape(-1, 1))
)
distances, indices = treated_neighbors.kneighbors(test['p1'].values.reshape(-1, 1))
delta_r = delta[indices.reshape(-1)]
influence = ipse.influence_function(test, treatment, cov, outcome, features, delta_r, model_y, model_t)
means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)
mae_incre = utils.abs_ate(true_effect, means_incre)
mae_incre

In [ ]:
delta_re = delta.numpy()
influence = ipse.influence_function(train_data, treatment, cov, outcome, features, delta_re, model_y, model_t)
means_incre, stds_incre = np.mean(influence, axis=0), sem(influence, axis=0)

print("Estimation effect {:.2f}".format(means_incre))
print("MAE {:.2f}".format(utils.abs_ate(true_effect, means_incre)))